In [1]:
from MyCreds.mycreds import USCensusAPI
import requests
import pandas as pd

In [27]:
# API Reference: https://www.census.gov/data/developers/guidance/api-user-guide.Example_API_Queries.html
host = 'https://api.census.gov/data'
year = '/2019'
# Data Dictionary: https://api.census.gov/data.html
dataset_acronym = '/acs/acs1'
g = '?get='
# Variables for the ACS: https://api.census.gov/data/2005/acs/acs1/variables.html
variables = 'NAME,B01001_001E'
location = '&for=us:*'
usr_key = f"&key={USCensusAPI.api_key}"

query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"

In [28]:
response = requests.get(query_url)

In [29]:
print(response.text)

[["NAME","B01001_001E","us"],
["United States","328239523","1"]]


B01001_001E is Estimated Total: Sex by Age without delineation.  In other words, this query is basically just returning 328,239,523 which is the total estimated US population in 2019.